# Working with earthquake and volcano data in prep for creating a Shiny app

Downloaded from Kaggle : 
- https://www.kaggle.com/datasets/thedevastator/uncovering-geophysical-insights-analyzing-usgs-e
- https://www.kaggle.com/datasets/jessemostipak/volcano-eruptions

Found though Google's dataset search : https://datasetsearch.research.google.com/

And starting from this example : https://shinylive.io/py/examples/#map


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

## Earthquakes

https://www.kaggle.com/datasets/thedevastator/uncovering-geophysical-insights-analyzing-usgs-e

In [ ]:
edf = pd.read_csv('data/earthquakes/usgs_main.csv')
edf['time'] = pd.to_datetime(edf['time'])
# limit to ~150 events for ipyleaflet
#edf = edf.loc[edf['time'] >= pd.to_datetime('2022-09-01T00:00:00.00Z')]
#edf = edf.loc[edf['mag'] > 5.7]
#edf = edf.loc[edf['mag'] > 1]
edf

In [ ]:
edf.columns

In [ ]:
print(np.min(edf['time']), np.max(edf['time']))

In [ ]:
# create some normalized values for scaling points on the maps
edf['normDepth'] = np.nan_to_num((edf['depth'] - np.amin(edf['depth']))/(np.amax(edf['depth']) - np.amin(edf['depth'])), 0)#*30 + 0.1
edf['normMag'] = np.nan_to_num(10.**(3*(edf['mag'] - np.amin(edf['mag']))/(np.amax(edf['mag']) - np.amin(edf['mag']))),0)

In [ ]:
f, ax = plt.subplots()
_ = ax.hist(edf['normMag'], bins=50)

In [ ]:
f, ax = plt.subplots()
ax.scatter(edf['normMag'], edf['normDepth'])

## Volcanoes 

https://www.kaggle.com/datasets/jessemostipak/volcano-eruptions

In [ ]:
vdf = pd.read_csv('data/volcanos/volcano.csv')
# limit to ~150 rows for ipyleaflet
vdf['last_eruption_year'] = vdf['last_eruption_year'].replace('Unknown', np.nan).astype('float')
#vdf = vdf.loc[vdf['last_eruption_year'] >= 2000]
vdf

In [ ]:
vdf.columns

In [ ]:
# create some normalized values for scaling points on the maps
vdf['normPop'] = (vdf['population_within_100_km'] - np.amin(vdf['population_within_100_km']))/(np.amax(vdf['population_within_100_km']) - np.amin(vdf['population_within_100_km']))#**3.#*300 + 1
vdf['normYear'] = (vdf['last_eruption_year'] - np.amin(vdf['last_eruption_year']))/(np.amax(vdf['last_eruption_year']) - np.amin(vdf['last_eruption_year']))#**3.#*300 + 1
vdf['normElevation'] = (vdf['elevation'] - np.amin(vdf['elevation']))/(np.amax(vdf['elevation']) - np.amin(vdf['elevation']))#**3.#*300 + 1

In [ ]:
f, ax = plt.subplots()
_ = ax.hist(vdf['population_within_100_km'])

# Create the map with ipyleaflet

Note this will only accommodate ~150 points for each category

In [ ]:
from ipyleaflet import Map, CircleMarker, LayersControl, LayerGroup, basemaps

In [ ]:
def createMap(eSizeCol = 'normMag', vSizeCol = 'normPop'):
    m = Map(basemap = basemaps.CartoDB.DarkMatter, center = (42, 0), zoom = 1.5)

    # earthquakes
    earthquakes = []
    for i, row in edf.iterrows():

        circle_marker = CircleMarker(
            title = row['place'],
            location = (row['latitude'], row['longitude']),
            radius = int(np.round(row[eSizeCol])),
            weight = 1,
            color = "#0d6aff", 
            fill_color = "#0d6aff",
        )
        earthquakes.append(circle_marker)


    # volcanoes
    volcanoes = []
    for i, row in vdf.iterrows():

        circle_marker = CircleMarker(
            title = row['volcano_name'],
            location = (row['latitude'], row['longitude']),
            radius = int(np.round(row[vSizeCol])),
            weight = 1,
            color = "#ff1d0d", 
            fill_color = "#ff1d0d",
        )
        volcanoes.append(circle_marker)

    earthquake_layer_group = LayerGroup(layers = earthquakes, name = "Earthquakes")
    volcano_layer_group = LayerGroup(layers = volcanoes, name = "Volcanos")
    m.add_layer(earthquake_layer_group)
    m.add_layer(volcano_layer_group)

    control = LayersControl(position = 'topright')
    m.add_control(control)
    
    return m

In [ ]:
m = createMap(eSizeCol = 'normDepth', vSizeCol = 'normElevation')
m

## This looks great and works in Shiny, but (initially) not in shinyapps.io

shinyapps.io does not have Python 3.10, and for some reason this doesn't work with 3.9.13 in shiny.  So... I will try with a cartopy map.  

I created a separate conda environment with Python 3.9.13 to work on this: `shinytest3.9.13-wsl`.

(Later, I discovered the issue with shiny widgets, see below, but other tools will allow me to plot more markers.)

# Create the map with cartopy

This will allow for any number of points, but the map is static.

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
def createMap(f, eSizeCol = 'normMag', vSizeCol = 'normPop'):
    ax = f.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    ax.coastlines()
    ax.add_feature(cfeature.LAND, color = 'black')
    ax.add_feature(cfeature.OCEAN, facecolor = 'gray')
    ax.add_feature(cfeature.LAKES, color = 'gray')

    ax.scatter(edf['longitude'], edf['latitude'], marker = 'o',
                edgecolor = [13/255., 106/255., 255/255., 1.], facecolor = [13/255., 106/255., 255/255., 0.2], 
                s = edf[eSizeCol],
                transform = ccrs.Geodetic(), zorder = 3)

    ax.scatter(vdf['longitude'], vdf['latitude'], marker = 'o',
                edgecolor = [255/255., 29/255., 13/255., 1.], facecolor = [255/255., 29/255., 13/255., 0.2], 
                s = vdf[vSizeCol], 
                transform = ccrs.Geodetic(), zorder = 3)


In [ ]:
f = plt.figure(figsize = (10,10))
createMap(f)

## This looks very nice!  But I miss the interactivity (zoom, pan, etc.)

# Try with Plotly

Examples
- https://plotly.com/python/mapbox-layers/
- https://plotly.com/python/scattermapbox/

This works great here, but... I get the same error in Shiny with python 3.9.13 as with ipyleaflet.  So this is an error with shinywidgets.

Wait!! I found the issue : https://github.com/rstudio/py-shinywidgets/issues/79

I need to downgrade the veresion of ipywidgets, and that fixes this issue!

```
conda install ipywidgets==7.7.3
```

## Plotly express offers an easy "one-liner"

But if I wanted to plot more than one data set (earthquakes and volcanoes), I would need to combine them into a single `pandas DataFrame`.

In [ ]:
import plotly.express as px

In [ ]:
# combine the dataframe so that I can use plotly express

fig = px.scatter_mapbox(edf, lat = "latitude", lon = "longitude", size = "normMag", 
                        color_discrete_sequence = ["#0d6aff"],
                        hover_name = "place", hover_data = ["mag", "depth"],
                        height = 700, zoom = 1)
fig.update_layout(mapbox_style = "carto-darkmatter")
fig.update_layout(margin = {"r":0,"t":0,"l":0,"b":0})
fig.show()

## Plotly graph objects allows for more flexilibity

Plus it will more easily allow me to change the points later on (e.g., to change how they are scaled and show/hide them in Shiny).

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scattermapbox(
        lat = edf['latitude'],
        lon = edf['longitude'],
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = edf['normDepth'].to_numpy()*300,
            sizemode = 'area',
            color = '#0d6aff',
            opacity = 0.7
        ),
        text = edf['place'] + '<br>Magnitude:' + edf['mag'].astype('str'),
        hoverinfo = 'text'
    )
)

fig.add_trace(
    go.Scattermapbox(
        lat = vdf['latitude'],
        lon = vdf['longitude'],
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = vdf['normElevation'].to_numpy()**3.*300,
            sizemode = 'area',
            color = '#ff1d0d',
            opacity = 0.7
        ),
        text = vdf['volcano_name'],
        hoverinfo = 'text'
    )
)

fig.update_layout(
    autosize = True,
    hovermode = 'closest',
    height = 600,
    width = 1000,
    margin = {"r":0,"t":0,"l":0,"b":0},
    mapbox = dict(
        style = 'carto-darkmatter',
        zoom = 0.9
    ),
)

fig.show()

# Looking good!